### Fetch data

#### Load dependencies

In [1]:
# path variables
import sys
sys.path.insert(0, '/Users/sarfrazahmed/Documents/projects/deposit-model/conf')
from config import SQLQuery

# core libraries
import pandas as pd
import numpy as np

# plotting libraries
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
from pandas_profiling import ProfileReport

# setting constants
pio.templates.default = 'plotly_dark'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
color_map = {'high': '#00cc96', 'low': '#636efa'}

### Fetching Data

In [4]:
querySno = SQLQuery('snowflake')
q = querySno

df_raw = q("""with 

REVENUE AS 

(
select 
BUSINESS_ID,Abs(LAST_12M_CARD_REV)+Abs(LAST_12M_NET_FEES_REV) as LAST_12M_REV
from "PROD_DB"."BI"."BUSINESS_STATS"
),

TRANSACTIONS AS -- get all credit transactions
(
    select BUSINESS_ID,
           TRANSACTION_DATE,
           AMOUNT 
  from "PROD_DB"."DATA"."TRANSACTIONS" 
  where type='credit' and status='active'
  group by 1,2,3
),

BUSINESS_DETAIL as -- get all business accounts created between Jan 2021 and Feb 2022
(
  select *
  from "PROD_DB"."DATA"."BUSINESSES"
  where ACCOUNT_CREATE_DATE between '2021-01-01' and '2021-05-31'
),

ALLOY_PERSONS as -- get all the alloy persons data (max of application_versions_id in case of duplicates)
(
  select * from (
select * ,
       row_number() over (partition by APPLICATION_ID order by APPLICATION_VERSION_ID desc) as rank
from "PROD_DB"."DATA"."ALLOY_EVALUATIONS_PERSONS") 
where rank=1
),

APPLICATION_DETAIL as -- get all entries from applications table
(
  select *
  from "PROD_DB"."DATA"."APPLICATIONS"
),

BASE AS -- create base table with business and transactions
(
    select b.BUSINESS_ID,b.APPLICATION_ID,sum(t.amount) as total_credit_amount
    from BUSINESS_DETAIL b 
    inner join TRANSACTIONS t
    on b.BUSINESS_ID=t.BUSINESS_ID
    where DATEDIFF(day, ACCOUNT_CREATE_DATE, TRANSACTION_DATE) between 0 AND 60
    group by 1,2
)

    select base.*,R.LAST_12M_REV,B.*
    from base
    inner join REVENUE R
    on base.BUSINESS_ID=R.BUSINESS_ID
    left join APPLICATION_DETAIL B
    on base.APPLICATION_ID = B.APPLICATION_ID

""")

df_raw.shape

(55975, 102)

#### Add target variable

In [5]:
df = df_raw.copy(deep=True)
df['deposit_class'] = np.where(df['total_credit_amount']<800, 'low', np.where(df['total_credit_amount']>1200, 'high', 'mid'))
df = df[df['deposit_class'] != 'mid']
print(df['deposit_class'].value_counts())
df.shape

high    27719
low     24416
Name: deposit_class, dtype: int64


(52135, 103)

#### Deposit Class Distribution

In [6]:
fig = px.histogram(df, 
                   x="deposit_class", 
                   color="deposit_class", 
                   text_auto=True, 
                   color_discrete_map=color_map)
fig.show()

In [7]:
df[df['deposit_class'] == 'high']['last_12m_rev'].sum()

191555286.35

In [8]:
df[df['deposit_class'] == 'low']['last_12m_rev'].sum()

32188262.27

In [9]:
df[df['deposit_class'] == 'high']['last_12m_rev'].sum()/df['last_12m_rev'].sum()

0.8561376966239698

In [2]:
querySno = SQLQuery('snowflake')
q = querySno

df_raw = q("""with 

REV AS -- get all credit transactions
(
    select BUSINESS_ID,
           TRANSACTION_DATE,
           Abs(AMOUNT)*0.0185 as REVENUE 
  from "PROD_DB"."DATA"."TRANSACTIONS" 
  where type='debit' and status='active' and medium='POS Withdrawal'
  group by 1,2,3
),

BUSINESS_DETAIL as -- get all business accounts created between Jan 2021 and Feb 2022
(
  select *
  from "PROD_DB"."DATA"."BUSINESSES"
  where ACCOUNT_CREATE_DATE between '2021-01-01' and '2021-05-31'
),


APPLICATION_DETAIL as -- get all entries from applications table
(
  select *
  from "PROD_DB"."DATA"."APPLICATIONS"
),

BASE AS -- create base table with business and transactions
(
    select b.BUSINESS_ID,b.APPLICATION_ID,sum(t.REVENUE) as total_revenue
    from BUSINESS_DETAIL b 
    inner join REV t
    on b.BUSINESS_ID=t.BUSINESS_ID
    where DATEDIFF(day, ACCOUNT_CREATE_DATE, TRANSACTION_DATE) between 0 AND 365
    group by 1,2
),

TRANSACTIONS AS -- get all credit transactions
(
    select BUSINESS_ID,
           TRANSACTION_DATE,
           AMOUNT 
  from "PROD_DB"."DATA"."TRANSACTIONS" 
  where type='credit' and status='active'
  group by 1,2,3
),

FINAL_TRANSACTIONS AS 
(
  select b.BUSINESS_ID,b.APPLICATION_ID,sum(t.AMOUNT) as total_credit_amount
    from BUSINESS_DETAIL b 
    inner join TRANSACTIONS t
    on b.BUSINESS_ID=t.BUSINESS_ID
    where DATEDIFF(day, ACCOUNT_CREATE_DATE, TRANSACTION_DATE) between 0 AND 60
    group by 1,2
 )

    select base.*,FINAL_TRANSACTIONS.total_credit_amount,APPLICATION_DETAIL.*
    from base
    inner join FINAL_TRANSACTIONS
    on base.APPLICATION_ID = FINAL_TRANSACTIONS.APPLICATION_ID
    left join APPLICATION_DETAIL
    on base.APPLICATION_ID = APPLICATION_DETAIL.APPLICATION_ID
""")

df_raw.shape

(38501, 102)

In [3]:
df = df_raw.copy(deep=True)
df['deposit_class'] = np.where(df['total_credit_amount']<800, 'low', np.where(df['total_credit_amount']>1200, 'high', 'mid'))
df = df[df['deposit_class'] != 'mid']
print(df['deposit_class'].value_counts())
df.shape

high    20616
low     14959
Name: deposit_class, dtype: int64


(35575, 103)

In [4]:
len(df[df['deposit_class'] == 'high'])/len(df)

0.5436301401978713

In [4]:
df[df['deposit_class'] == 'high']['total_revenue'].sum()/df['total_revenue'].sum()

0.8831494346591499

In [5]:
df[df['deposit_class'] == 'low']['total_revenue'].sum()/df['total_revenue'].sum()

0.11685056534085027

In [6]:
df[df['deposit_class'] == 'high']['total_revenue'].sum()

4184696.08651

In [7]:
df[df['deposit_class'] == 'low']['total_revenue'].sum()

553682.17914

In [8]:
df[df['deposit_class'] == 'high']['azlo_referral']

0         True
1         True
2        False
3        False
4        False
         ...  
38474     True
38481    False
38493    False
38494    False
38495    False
Name: azlo_referral, Length: 20616, dtype: bool

In [9]:
pd.crosstab(df.azlo_referral, df.deposit_class)

deposit_class,high,low
azlo_referral,,
False,12665,11263
True,7951,3696


In [13]:
df['total_revenue'].median()

34.18208

In [4]:
fig = px.histogram(df, 
                   x="deposit_class",
                   y="total_revenue",
                   color="deposit_class", 
                   text_auto='.2s',
                   color_discrete_map=color_map,
                   category_orders={'deposit_class': ['low', 'high']},)
fig.update_layout(
    width=1100,
    height=500
)
fig.show()